In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import math
from numpy.random import multinomial

In [2]:
np.random.seed(0)

#  Data generation

In [3]:
#networks simulation
N = 100
D0 = np.random.randint(2,size=(N,N))
D1 = np.random.randint(2,size=(N,N))
for i in range(N):
    D0[i,i] = 0
    D1[i,i] = 0

In [4]:
M0 = np.sum(D0,axis=1)
G0 = np.linalg.solve(np.diag(M0),D0)

M1 = np.sum(D1,axis=1)
G1 = np.linalg.solve(np.diag(M1),D1)

In [5]:
F0 = (np.dot(D0,D0)>0).astype(np.int)
F1 = (np.dot(D1,D1)>0).astype(np.int)

In [6]:
#GPA simulation
gpa_0 = np.random.uniform(low= 0,high=4,size=N)
gpa_1 = np.random.uniform(low=0,high=4,size=N)

# 1. MLIM model

On fait la régression bayésienne, avec les a posteriori conjugués (inverse-$\chi^2(1)$, normale conditionnelle).

si on veut juste les means et vars, il suffit de les calculer les paramètres des posterior.

In [7]:
#priors 
#on a 4 coefficients beta sur lesquels travailler 
mu0 = np.zeros(4) #mean of the conditional normal distribution 
Q0 = np.diag(np.ones(4)) #var of the conditional normal distribution
a0 = 1 #coef of the inverse gamma
b0 = 1 #scaling coefficient of the inverse gamma

In [8]:
#construction des variables explicatives et variables cibles
Y = gpa_1
Ybar = G1.dot(Y)
X = gpa_0
Xbar = G1.dot(X)
features = np.column_stack((np.ones(len(Y)),Ybar,X,Xbar))

Les paramètres du modèle sont mis à jour selon les équations suivantes :
$$ Q_{n}=(\mathbf {X} ^{\rm {T}}\mathbf {X} + Q_{0}) $$
$$  \boldsymbol\mu_n=(\mathbf{X}^{\rm T}\mathbf{X}+ Q_0)^{-1} (Q_0\boldsymbol\mu_0+\mathbf{X}^{\rm T}\mathbf{y})  $$ 
$$ a_{n}=a_{0}+{\frac  {n}{2}} $$ 
$$ b_{n}=b_{0}+{\frac  {1}{2}}({\mathbf  {y}}^{{{\rm {T}}}}{\mathbf  {y}}+{\boldsymbol  \mu }_{0}^{{{\rm {T}}}}Q_{0}{\boldsymbol  \mu }_{0}-{\boldsymbol  \mu }_{n}^{{{\rm {T}}}}Q_{n}{\boldsymbol  \mu }_{n}) $$

In [9]:
#updating parameters: computing the posterior distribution parameters
Qn = features.T.dot(features) + Q0
mun = np.linalg.solve(Qn,Q0.dot(mu0)+features.T.dot(Y))
an = a0+len(Y)/2
bn = b0 + 0.5*(Y.dot(Y)+mu0.T.dot(Q0.dot(mu0))-mun.T.dot(Qn.dot(mun)))

In [10]:
#moyennes des lois normales a posteriori
mun

array([ 1.46692776,  0.42632134,  0.16548153, -0.35156051])

On obtient donc 
$$ \beta_0 = 1.46 , \beta_{\bar{Y}} = 0.42, \beta_x = 0.16, \beta_{\bar{x}} = -0.35 $$

In [11]:
#matrice de variance covariance
Qn

array([[101.        , 192.73912912, 198.45645215, 199.66130908],
       [192.73912912, 373.77063606, 380.04142165, 385.00817177],
       [198.45645215, 380.04142165, 545.17215733, 392.69089827],
       [199.66130908, 385.00817177, 392.69089827, 401.00271555]])

Ce résultat est étrange

In [12]:
bn/(an-1) #moyenne de l'inverse-gamma

1.2081525826648845

In [13]:
bn**2/((an-1)**2*(an-2)) #variance de l'inverse gamma

0.0297884216938741

# 2. Exogenous network formation

$\alpha \sim \mathcal{N}(0,1)$ (chaque \alpha suit une loi normale de façon indépendante)

$P(D_{ij} = 1|D_{0},X)  = \frac{e^{\alpha_{0} + \alpha_{x}|Xi-Xj| + \alpha_{d}D_{0ij}+ \alpha_{f}F_{0ij}}}{1 + e^{\alpha_{0} + \alpha_{x}|Xi-Xj| + \alpha_{d}D_{0ij}+ \alpha_{f}F_{0ij}}}$

=> cas conjugué?

à déterminer: posterior (à calculer)

In [14]:
#priors
alpha0 = np.random.randn(1)
alphax = np.random.randn(1)
alphad = np.random.randn(1)
alphaf = np.random.randn(1)

In [16]:
#feature engineering 
#we must create |Xi-Xj| for each (i,j) couple 
dist = np.zeros((N,N))
for i in range(N):
    for j in range(i,N):
        value = np.abs(X[i]-X[j])
        dist[i][j] = value
        dist[j][i] = value
#We create a new D1 matrix based on the probability formula
D1_new = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        U = np.exp(alpha0 + alphax*dist[i][j] + alphad*D0[i][j] + alphaf*F0[i][j])
        p = (U)/(1 + U)
        D1_new[i][j] = np.random.binomial(1,p**2)

In [57]:
#updates


# 3. Endogenous network formation

aller voir appendix

# 4. Heterogeneity in peer effects

Same as 1. with different networks and extended models